In [1]:
import json
import pandas as pd
import urllib.request as rq

## Getting the latest covid data in a usable format

In [3]:
LATEST_URL = "https://covid.ourworldindata.org/data/latest/owid-covid-latest.json"

with rq.urlopen(LATEST_URL) as url:
    raw_data = json.loads(url.read().decode())

In [4]:
raw_df = pd.DataFrame(raw_data)

In [6]:
transpose_df = raw_df.transpose()

## Getting the whole covid data in a usable format

In [7]:
transpose_df.head()

,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
AFG,Asia,Afghanistan,2022-11-03,203395.0,130.0,112.429,7825.0,2.0,0.714,4945.321,...,None,37.746,0.5,64.83,0.511,41128772.0,None,None,None,None
OWID_AFR,None,Africa,2022-11-03,12388153.0,894.0,908.286,256982.0,1.0,8.143,8682.859,...,None,None,None,None,None,1426736614.0,None,None,None,None
ALB,Europe,Albania,2022-11-03,332996.0,0.0,15.286,3593.0,0.0,0.0,117156.49,...,51.2,None,2.89,78.57,0.795,2842318.0,None,None,None,None
DZA,Africa,Algeria,2022-11-03,270847.0,7.0,4.286,6881.0,0.0,0.0,6031.794,...,30.4,83.741,1.9,76.88,0.748,44903228.0,None,None,None,None
AND,Europe,Andorra,2022-11-03,46588.0,0.0,7.571,155.0,0.0,0.0,583495.109,...,37.8,None,None,83.73,0.868,79843.0,None,None,None,None


In [19]:
import pyodbc

server = 'FETTUCINE' 
database = 'prefect_demo' 
cnxn = pyodbc.connect(r'Driver=SQL Server;Server=FETTUCCINE;Database=prefect_demo;Trusted_Connection=yes')
# cnxn = pyodbc.connect('Trusted_Connection=yes',driver='{SQL Server}',server='.\FETTUCCINE',database='prefect_demo')

In [22]:
cnxn.close()

In [25]:
import sqlalchemy


engine = sqlalchemy.create_engine(
               "mssql+pyodbc://FETTUCCINE/prefect_demo?driver=SQL+Server",
               echo=False)

In [27]:
smaller = transpose_df[['location', 'total_cases', 'new_cases', 'total_deaths']]
smaller.reset_index(drop=True, inplace=True)
smaller.head()

,location,total_cases,new_cases,total_deaths
0,Afghanistan,203395.0,130.0,7825.0
1,Africa,12388153.0,894.0,256982.0
2,Albania,332996.0,0.0,3593.0
3,Algeria,270847.0,7.0,6881.0
4,Andorra,46588.0,0.0,155.0


In [28]:
with engine.connect() as conn:
    smaller.to_sql(name='country_cases', con=conn, if_exists='replace')